In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from tensorflow import layers, models
import tensorflow_model_optimization as tfmot

print("TensorFlow version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "not available")


In [ ]:
def create_synthetic_database():
    # create synthectis database for demonstration

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    class_names =['plastic','paper', 'glass']

    x_train = x_train[:3000] / 255.0
    y_train = y_train[:3000] % 3

    x_test = x_test[:500] / 255.0
    y_test = y_test[:500] % 3

    y_train = tf.keras.utils.to_categorical(y_train,3)
    y_test = tf.keras.utils.to_categorical(y_test,3)

    return (x_train, y_train), (x_test, y_test), class_names
(x_train, y_train), (x_test, y_test), class_names = create_synthetic_database()

print(f"Training data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Classes: {class_names}")


In [ ]:
# Visualize some samples from the dataset
plt.figure(figsize=(12, 8))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(x_train[i])
    plt.title(f'Class: {class_names[np.argmax(y_train[i])]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# model creation
def create_edge_model(input_shape=(32, 32, 3), num_classes=3):
    """
    Create a lightweight CNN model optimized for edge devices
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create and display model
model = create_edge_model()
model.summary()


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
print("Starting model training...")
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=5, 
    validation_data=(x_test, y_test),
    verbose=1
)